In [9]:
import requests
import random
from collections import Counter
import numpy as np


Acima eu fiz apenas os imports necessarios:

requests para o download do vocabulario

random para sortear a palavra

collections,usei a funcao counter para contar a frequencia

numpy usei para calculos 

Classe `JogoDeForca`

A célula abaixo define a classe `JogoDeForca`, que gerencia o jogo da forca:
- O método `novo_jogo` escolhe uma palavra aleatória e define o número de vidas.
- O método `tentar_letra` permite ao jogador tentar adivinhar uma letra. Se a letra estiver errada, uma vida é perdida.
- O método `tentar_palavra` permite ao jogador tentar adivinhar a palavra completa. Se o jogador errar, ele perde todas as vidas restantes.

Essa classe é responsável por controlar as regras básicas do jogo.


In [10]:
class JogoDeForca:
    def __init__(self):
        # Baixa o vocabulário de palavras
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        self.content = str(r.content.decode()).split('\n') if r.status_code == 200 else []
            
    def novo_jogo(self, vidas=5):
        # Inicializa um novo jogo e escolhe uma palavra aleatoriamente
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        return len(self.palavra)
    
    def tentar_letra(self, letra):
        # Tenta adivinhar uma letra na palavra, reduzindo vidas se errada
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx] == letra]
            else:
                self.vidas -= 1
                return []
        return False

    def tentar_palavra(self, palavra):
        # Verifica se a palavra completa está correta
        if self.vidas > 0:
            if self.palavra == palavra:
                return True
            else:
                self.vidas = 0
                return False


# Classe `JogadorDeForca`

Esta célula define a classe `JogadorDeForca`, que implementa a lógica do jogador automático:
- O método `calcular_frequencia_letras` calcula a frequência de cada letra no vocabulário.
- O método `escolher_proxima_letra` escolhe a próxima letra com base na frequência relativa das letras que ainda não foram tentadas.
- O método `atualizar_palavras_possiveis` ajusta a lista de palavras possíveis com base no feedback obtido, eliminando palavras que não se alinham com o resultado das tentativas anteriores.

Essa classe representa a estratégia automática do jogador, baseada em entropia e probabilidade, priorizando letras mais comuns para aumentar a chance de acerto.


In [11]:
class JogadorDeForca:
    def __init__(self, vocabulario):
        # Inicializa o jogador com o vocabulário e calcula a frequência das letras
        self.vocabulario = vocabulario
        self.frequencias = self.calcular_frequencia_letras()
        self.resetar_jogo()

    def calcular_frequencia_letras(self):
        # Calcula a frequência relativa de cada letra no vocabulário
        letras = ''.join(self.vocabulario)
        frequencias = Counter(letras)
        total = sum(frequencias.values())
        return {letra: freq / total for letra, freq in frequencias.items()}

    def resetar_jogo(self):
        # Reinicia o estado do jogador para um novo jogo
        self.vidas = 5
        self.letras_tentadas = set()
        self.palavras_possiveis = self.vocabulario

    def escolher_proxima_letra(self):
        # Escolhe a letra com maior frequência entre as não tentadas
        frequencias_filtradas = {letra: freq for letra, freq in self.frequencias.items() if letra not in self.letras_tentadas}
        return max(frequencias_filtradas, key=frequencias_filtradas.get, default=None)

    def atualizar_palavras_possiveis(self, letra, posicoes):
        # Atualiza as palavras possíveis com base nas posições da letra
        if posicoes:
            self.palavras_possiveis = [palavra for palavra in self.palavras_possiveis if all(w == letra for i, w in enumerate(palavra) if i in posicoes)]
        else:
            self.palavras_possiveis = [palavra for palavra in self.palavras_possiveis if letra not in palavra]

    def jogar(self, jogo):
        # Loop principal do jogo para o jogador automático
        tamanho_palavra = jogo.novo_jogo(self.vidas)
        palavra_descoberta = ['_'] * tamanho_palavra
        
        while self.vidas > 0 and '_' in palavra_descoberta:
            letra = self.escolher_proxima_letra()
            if letra is None:
                break
            
            self.letras_tentadas.add(letra)
            posicoes = jogo.tentar_letra(letra)
            if posicoes is False:
                return False
            
            for pos in posicoes:
                palavra_descoberta[pos] = letra
            
            self.atualizar_palavras_possiveis(letra, posicoes)
            if '_' not in palavra_descoberta:
                return jogo.tentar_palavra(''.join(palavra_descoberta))
        
        return False


# Avaliação do Jogador

Nesta célula abaixo, a função `avaliar_jogador` executa 100 partidas para medir o desempenho do jogador automático.
- A função exibe o resultado de cada partida (vitória ou derrota).
- No final, calcula e exibe a taxa de vitórias para avaliar a eficácia da estratégia de entropia.

Esta avaliação fornece uma métrica para entender quão bem o jogador automático se sai em múltiplas partidas.


In [12]:
def avaliar_jogador():
    # Executa múltiplas partidas e calcula a taxa de vitória
    jogo = JogoDeForca()
    jogador = JogadorDeForca(jogo.content)
    vitorias = 0
    total_jogos = 100
    
    for i in range(total_jogos):
        jogador.resetar_jogo()
        resultado = jogador.jogar(jogo)
        
        if resultado:
            print(f"Jogo {i + 1}: Vitória!")
            vitorias += 1
        else:
            print(f"Jogo {i + 1}: Derrota.")
    
    print(f"\nTaxa de vitória final: {vitorias / total_jogos * 100:.2f}%")

# Executar a avaliação do jogador automático
avaliar_jogador()


Jogo 1: Derrota.
Jogo 2: Derrota.
Jogo 3: Derrota.
Jogo 4: Derrota.
Jogo 5: Derrota.
Jogo 6: Derrota.
Jogo 7: Derrota.
Jogo 8: Derrota.
Jogo 9: Derrota.
Jogo 10: Derrota.
Jogo 11: Derrota.
Jogo 12: Derrota.
Jogo 13: Derrota.
Jogo 14: Vitória!
Jogo 15: Derrota.
Jogo 16: Derrota.
Jogo 17: Derrota.
Jogo 18: Derrota.
Jogo 19: Derrota.
Jogo 20: Derrota.
Jogo 21: Vitória!
Jogo 22: Derrota.
Jogo 23: Derrota.
Jogo 24: Derrota.
Jogo 25: Vitória!
Jogo 26: Derrota.
Jogo 27: Derrota.
Jogo 28: Vitória!
Jogo 29: Derrota.
Jogo 30: Derrota.
Jogo 31: Vitória!
Jogo 32: Derrota.
Jogo 33: Derrota.
Jogo 34: Derrota.
Jogo 35: Derrota.
Jogo 36: Derrota.
Jogo 37: Derrota.
Jogo 38: Vitória!
Jogo 39: Derrota.
Jogo 40: Derrota.
Jogo 41: Vitória!
Jogo 42: Derrota.
Jogo 43: Derrota.
Jogo 44: Derrota.
Jogo 45: Derrota.
Jogo 46: Vitória!
Jogo 47: Derrota.
Jogo 48: Derrota.
Jogo 49: Derrota.
Jogo 50: Derrota.
Jogo 51: Vitória!
Jogo 52: Derrota.
Jogo 53: Derrota.
Jogo 54: Derrota.
Jogo 55: Derrota.
Jogo 56: Derrota.
J

# Resultados da Avaliacao



Após a execução de 100 partidas, os resultados mostram que o jogador automático obteve uma taxa de vitória de 14.00%. Isso significa que, em aproximadamente 14% das tentativas, o jogador conseguiu adivinhar corretamente a palavra dentro do limite de cinco vidas.

### Interpretação dos Resultados

A estratégia baseada em entropia e frequência de letras demonstrou alguma eficácia, mas também possui limitações. A taxa de vitória não é muito alta porque:
1. **Limitação de Vidas**: Com apenas cinco vidas, o jogador tem um número restrito de tentativas, o que limita suas chances de explorar o vocabulário completo.
2. **Palavras com Letras Raras**: Palavras com letras menos frequentes podem não ser adivinhadas facilmente, já que o jogador prioriza letras mais comuns.
3. **Tamanho das Palavras**: Palavras longas e curtas podem exigir abordagens diferentes. Em algumas palavras, a incerteza inicial é alta, o que leva a erros nas primeiras tentativas.

### Conclusão

Embora a estratégia baseada em entropia aumente a probabilidade de sucesso ao priorizar letras comuns, ela ainda apresenta um desafio em acertar palavras com letras incomuns ou em partidas onde o limite de cinco vidas é insuficiente para identificar a palavra completa. Esse resultado demonstra a importância de ajustar a estratégia em função do vocabulário e das restrições do jogo.